# L2Q End-to-End Pipeline (Supplement)

This notebook reproduces the paper's pipeline: **Lecture PDF → local LLM MCQ generation → deterministic QC → JSON/CSV export**.

**Offline** here means: the lecture content is **not sent to any external LLM service**.

Notes:
- If you run on Google Colab, computation happens on a remote machine (Colab), but the pipeline still avoids sending lecture text to external LLM APIs.
- For local execution, replace `/content/...` paths as needed.

Quick start (Colab): upload `arxiv_dummy_entropy_pack.zip`, enable GPU (optional), then `Runtime → Run all`.


In [ ]:
!nvidia-smi


In [ ]:
import os, zipfile

ZIP_PATH = "/content/arxiv_dummy_entropy_pack.zip"
PACK_DIR = "/content/arxiv_dummy_entropy_pack"

if os.path.exists(PACK_DIR):
    print("Already extracted:", PACK_DIR)
else:
    if not os.path.exists(ZIP_PATH):
        raise FileNotFoundError("ZIP file not found. Upload arxiv_dummy_entropy_pack.zip to the session (left file panel).")
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall("/content")
    print("Extracted:", PACK_DIR)

print("\n--- PACK CONTENTS (top) ---")
!ls -al "{PACK_DIR}" | head -n 50


In [ ]:
!apt-get -qq update
!apt-get -qq install -y git cmake build-essential

!pip -q install -U pymupdf pandas numpy jsonschema tqdm huggingface_hub


In [ ]:
import pathlib

LLAMA_DIR = pathlib.Path("/content/llama.cpp")
if not LLAMA_DIR.exists():
    !git clone -q https://github.com/ggml-org/llama.cpp /content/llama.cpp
else:
    print("llama.cpp exists:", LLAMA_DIR)

%cd /content/llama.cpp

!cmake -S . -B build -DGGML_CUDA=ON -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j

print("build done")


In [ ]:
import os, subprocess

CANDIDATES = [
    "/content/llama.cpp/build/bin/llama-cli",
    "/content/llama.cpp/build/bin/main",
    "/content/llama.cpp/llama-cli",
    "/content/llama.cpp/main",
]
LLAMA_BIN = next((p for p in CANDIDATES if os.path.exists(p)), None)
if LLAMA_BIN is None:
    raise FileNotFoundError("llama-cli / main not found. The build may have failed.")

print("✅ llama binary:", LLAMA_BIN)

try:
    HELP = subprocess.check_output([LLAMA_BIN, "--help"], text=True, errors="ignore")
except Exception:
    HELP = subprocess.check_output([LLAMA_BIN, "-h"], text=True, errors="ignore")

def supports(flag: str) -> bool:
    return flag in HELP

SUPPORT_JSON_SCHEMA_FILE  = supports("--json-schema-file") or supports("-jf")
SUPPORT_GRAMMAR_FILE      = supports("--grammar-file")
SUPPORT_NO_DISPLAY_PROMPT = supports("--no-display-prompt")
SUPPORT_NO_PERF           = supports("--no-perf")
SUPPORT_SIMPLE_IO         = supports("--simple-io")
SUPPORT_LOG_DISABLE       = supports("--log-disable")
SUPPORT_SINGLE_TURN       = supports("--single-turn") or supports("-st")

print("supports JSON schema file (-jf):", SUPPORT_JSON_SCHEMA_FILE)
print("supports grammar file:", SUPPORT_GRAMMAR_FILE)
print("supports --no-display-prompt:", SUPPORT_NO_DISPLAY_PROMPT)
print("supports --no-perf:", SUPPORT_NO_PERF)
print("supports --simple-io:", SUPPORT_SIMPLE_IO)
print("supports --log-disable:", SUPPORT_LOG_DISABLE)
print("supports single-turn (-st):", SUPPORT_SINGLE_TURN)


In [ ]:
# Configuration (edit as needed)

PDF_PATH = "/content/arxiv_dummy_entropy_pack/dummy_lecture_01_information_theory_entropy.pdf"
#PDF_PATH = "/content/arxiv_dummy_entropy_pack/dummy_lecture_02_thermodynamics_second_law_entropy.pdf"
#PDF_PATH = "/content/arxiv_dummy_entropy_pack/dummy_lecture_03_statistical_mechanics_entropy.pdf"

N_QUESTIONS = 8
N_CHOICES = 5

PDF_MAX_PAGES_TO_READ = None
PDF_PROGRESS_EVERY = 1
PDF_PER_PAGE_WARN_SEC = 6.0

RUN_MODE = "deploy"  # 'eval' or 'deploy'

USE_TOPIC_PLAN = True
CUSTOM_TOPIC_PLAN = []

MAX_TRIES_PER_QUESTION = 6
DUP_SIM_THRESHOLD = 0.92

ENABLE_PX_EQUIV_TEST = True
PX_EQUIV_TRIALS = 5

ENABLE_CONST_EQUIV_CHECK = True
CONST_EQUIV_TOL = 1e-9

ENABLE_UNIQUE_CORRECT_VERIFIER = False
VERIFIER_MAX_NEW_TOKENS = 220

MODEL_MODE = "hf_download"  # 'hf_download' or 'local_gguf'
HF_REPO_ID = "bartowski/Qwen2.5-14B-Instruct-GGUF"
HF_FILENAME = "Qwen2.5-14B-Instruct-Q4_K_M.gguf"
LOCAL_GGUF_PATH = "/content/models/your_model.gguf"

CTX_SIZE = 4096
N_GPU_LAYERS = 999
TEMPERATURE = 0.2
TOP_P = 0.9
SEED = 0
MAX_NEW_TOKENS = 900
RETRY_MAX = 3

RUN_BATCH = False
BATCH_PDF_LIST = [
    "/content/arxiv_dummy_entropy_pack/dummy_lecture_01_information_theory_entropy.pdf",
    "/content/arxiv_dummy_entropy_pack/dummy_lecture_02_thermodynamics_second_law_entropy.pdf",
    "/content/arxiv_dummy_entropy_pack/dummy_lecture_03_statistical_mechanics_entropy.pdf",
]
BATCH_SEEDS = [0]

print('PDF_PATH:', PDF_PATH)
print('RUN_MODE:', RUN_MODE)
print('MODEL_MODE:', MODEL_MODE)


In [ ]:
import os, pathlib
from huggingface_hub import hf_hub_download

MODELS_DIR = pathlib.Path("/content/models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

def ensure_model() -> str:
    if MODEL_MODE == "hf_download":
        print("⏬ Downloading GGUF from Hugging Face ...")
        local_path = hf_hub_download(
            repo_id=HF_REPO_ID,
            filename=HF_FILENAME,
            local_dir=str(MODELS_DIR),
            local_dir_use_symlinks=False,
        )
        print("✅ downloaded:", local_path)
        return local_path
    elif MODEL_MODE == "local_gguf":
        if not os.path.exists(LOCAL_GGUF_PATH):
            raise FileNotFoundError(
                f"LOCAL_GGUF_PATH not found: {LOCAL_GGUF_PATH}\n"
                "Upload your .gguf file to the session and set LOCAL_GGUF_PATH correctly."
            )
        print("✅ using local gguf:", LOCAL_GGUF_PATH)
        return LOCAL_GGUF_PATH
    else:
        raise ValueError("MODEL_MODE must be 'hf_download' or 'local_gguf'.")

MODEL_PATH = ensure_model()


In [ ]:
# ======================================
# ======================================
import fitz  # PyMuPDF
import os, re, time
from typing import List

if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(f"PDF not found: {PDF_PATH}")

base = os.path.basename(PDF_PATH)

print("Extracting PDF text…", flush=True)
t0 = time.time()

doc = fitz.open(PDF_PATH)
print(f"✅ opened: {base} | pages={doc.page_count}", flush=True)

max_pages = PDF_MAX_PAGES_TO_READ
n_pages_total = doc.page_count
n_pages_read = n_pages_total if (max_pages is None) else min(max_pages, n_pages_total)

pages = []
for i in range(n_pages_read):
    t_page = time.time()
    page = doc.load_page(i)

    try:
        txt = page.get_text("text")
    except Exception as e:
        print(f"⚠️ get_text('text') failed on p.{i+1}: {e} → fallback to 'blocks'", flush=True)
        blocks = page.get_text("blocks")
        txt = "\n".join([b[4] for b in blocks if isinstance(b, (list, tuple)) and len(b) >= 5])

    txt = re.sub(r"[ \t]+\n", "\n", txt)
    txt = re.sub(r"\n{3,}", "\n\n", txt)

    pages.append(f"(p.{i+1})\n{txt.strip()}")

    dt = time.time() - t_page
    if dt >= PDF_PER_PAGE_WARN_SEC:
        print(f"⚠️ p.{i+1}/{n_pages_read} extracted in {dt:.2f}s (slow page)", flush=True)

    if (i + 1) % max(1, PDF_PROGRESS_EVERY) == 0:
        print(f"… progress: {i+1}/{n_pages_read} pages | elapsed={(time.time()-t0):.1f}s", flush=True)

doc.close()

DOC_TEXT = "\n\n".join(pages).strip()

print("✅ pages(read/total):", n_pages_read, "/", n_pages_total, flush=True)
print("✅ text chars:", len(DOC_TEXT), flush=True)
print("\n--- preview (head) ---\n", flush=True)
print(DOC_TEXT[:1200], flush=True)

# -------------------------
# -------------------------
PRESET_TOPIC_PLANS = {
    "dummy_lecture_01_information_theory_entropy.pdf": [
        "Information content: I(x) = -log2 p(x)",
        "Shannon entropy definition: H(X) = -Σ p(x) log2 p(x) (bits)",
        "Non-negativity: H(X) ≥ 0 and when equality holds",
        "Maximum entropy: uniform distribution gives H(X)=log2 n",
        "Chain rule: H(X,Y)=H(X)+H(Y|X)",
        "Conditioning reduces entropy: H(X|Y) ≤ H(X)",
        "Worked example (biased coin): p(1)=0.9, p(0)=0.1 → H(X)≈0.469 bits",
        "Binary entropy function: maximum at p=0.5",
    ],
    "dummy_lecture_02_thermodynamics_second_law_entropy.pdf": [
        "Definition for reversible process: dS = δQ_rev / T",
        "Second law (Clausius statement): heat does not flow cold→hot spontaneously",
        "Second law (Kelvin–Planck): no cyclic engine converts all heat to work",
        "Entropy form of second law: isolated system has ΔS ≥ 0",
        "Clausius inequality for cycles: ∮(δQ/T) ≤ 0 and meaning of equality",
        "Entropy of universe: ΔS_universe = ΔS_system + ΔS_surroundings ≥ 0",
        "Worked example: reversible isothermal expansion ΔS = nR ln(V2/V1); compute for V2/V1=2",
        "Interpretation caution: 'disorder' vs microstates (high-level link)",
    ],
    "dummy_lecture_03_statistical_mechanics_entropy.pdf": [
        "Microstate vs macrostate: definitions and distinction",
        "Boltzmann entropy: S = k_B ln Ω (equal-probability microstates)",
        "Gibbs entropy: S = -k_B Σ p_i ln p_i (general case)",
        "Consistency check: Gibbs reduces to k_B ln Ω when p_i=1/Ω",
        "Canonical ensemble: p_i ∝ exp(-βE_i), β=1/(k_B T), partition function Z",
        "Free energy from partition function: F = -k_B T ln Z",
        "Entropy from free energy: S = - (∂F/∂T)_(V,N)",
        "Two-level system limits: S→0 at low T, S→k_B ln 2 at high T",
    ],
}

def auto_topic_plan_from_text(doc_text: str, n: int) -> List[str]:
    lines = [ln.strip() for ln in doc_text.splitlines()]
    cand = []

    for i, ln in enumerate(lines):
        if "learning objectives" in ln.lower():
            window = lines[i:i+60]
            for j in range(len(window)-1):
                a, b = window[j].strip(), window[j+1].strip()
                if re.fullmatch(r"\d+\.", a) and b and len(b) > 6:
                    cand.append(b)
                if re.match(r"^\d+\.", a) and len(a) > 6:
                    cand.append(a)

    for ln in lines:
        if ln.startswith("•") or ln.startswith("- "):
            cand.append(ln.lstrip("•- ").strip())

    for ln in lines:
        if re.search(r"worked example|summary|definition|properties|canonical ensemble|partition function", ln.lower()):
            if 6 <= len(ln) <= 120:
                cand.append(ln)

    out, seen = [], set()
    for c in cand:
        c2 = re.sub(r"\s+", " ", c).strip()
        if not c2:
            continue
        key = c2.lower()
        if key in seen:
            continue
        seen.add(key)
        out.append(c2)
        if len(out) >= n:
            break

    while len(out) < n:
        out.append(f"Key point #{len(out)+1} (auto)")

    return out[:n]

def get_topic_plan(pdf_path: str, doc_text: str, n: int) -> List[str]:
    base = os.path.basename(pdf_path)

    if CUSTOM_TOPIC_PLAN and isinstance(CUSTOM_TOPIC_PLAN, list) and len(CUSTOM_TOPIC_PLAN) > 0:
        tp = CUSTOM_TOPIC_PLAN[:]
        if len(tp) < n:
            tp += [f"Key point #{i+1} (custom fill)" for i in range(len(tp), n)]
        return tp[:n]

    if USE_TOPIC_PLAN and base in PRESET_TOPIC_PLANS:
        return PRESET_TOPIC_PLANS[base][:n]

    if USE_TOPIC_PLAN:
        return auto_topic_plan_from_text(doc_text, n)

    return ["" for _ in range(n)]

TOPIC_PLAN = get_topic_plan(PDF_PATH, DOC_TEXT, N_QUESTIONS)

print("\n✅ TOPIC_PLAN:")
for i, t in enumerate(TOPIC_PLAN, 1):
    print(f"  {i:02d}. {t}")

print(f"\nPDF extraction + topic plan: {(time.time()-t0):.2f}s", flush=True)


In [ ]:
import json, pathlib

SCHEMA_DIR = pathlib.Path("/content/schemas")
SCHEMA_DIR.mkdir(parents=True, exist_ok=True)
MCQ_SCHEMA_PATH = str(SCHEMA_DIR / "mcq_item_schema.json")

# NOTE:

mcq_schema = {
    "type": "object",
    "properties": {
        "ID": {"type": "string"},
        "Category": {"type": "string"},
        "SourcePages": {"type": "string"},
        "Question": {"type": "string"},
        "OptionA": {"type": "string"},
        "OptionB": {"type": "string"},
        "OptionC": {"type": "string"},
        "OptionD": {"type": "string"},
        "OptionE": {"type": "string"},
        "CorrectOption": {"type": "string"},
        "Explanation": {"type": "string"},
    },
    "required": ["Question","OptionA","OptionB","OptionC","OptionD","OptionE","CorrectOption","Explanation","SourcePages"],
}

with open(MCQ_SCHEMA_PATH, "w", encoding="utf-8") as f:
    json.dump(mcq_schema, f, ensure_ascii=False, indent=2)

print("✅ schema saved:", MCQ_SCHEMA_PATH)
print(json.dumps(mcq_schema, ensure_ascii=False, indent=2)[:800] + "\n...")

VERIFIER_SCHEMA_PATH = str(SCHEMA_DIR / "mcq_verifier_schema.json")
verifier_schema = {
    "type": "object",
    "properties": {
        "correct_options": {
            "type": "array",
            "items": {"type": "string"},
        },
        "notes": {"type": "string"},
    },
    "required": ["correct_options"],
}
with open(VERIFIER_SCHEMA_PATH, "w", encoding="utf-8") as f:
    json.dump(verifier_schema, f, ensure_ascii=False, indent=2)

print("✅ verifier schema saved:", VERIFIER_SCHEMA_PATH)


In [ ]:
import subprocess, time, json, re
from typing import Dict, Any, Tuple, Optional, List
from jsonschema import validate, ValidationError

def _extract_json_object(text: str) -> Optional[dict]:
    '''Extract the first JSON object from stdout and parse it (simple helper).'''
    t = (text or "").strip()
    t = re.sub(r"^```(json)?\s*", "", t)
    t = re.sub(r"\s*```$", "", t)
    if "{" in t and "}" in t:
        start = t.find("{")
        end = t.rfind("}")
        cand = t[start:end+1]
        try:
            return json.loads(cand)
        except Exception:
            cand2 = re.sub(r'\\(?!["\\\\/bfnrtu])', r"\\\\", cand)
            try:
                return json.loads(cand2)
            except Exception:
                return None
    return None

def llama_generate_json_one(
    prompt: str,
    model_path: str,
    schema_path: str,
    ctx_size: int = 4096,
    max_new_tokens: int = 900,
    temperature: float = 0.2,
    top_p: float = 0.9,
    seed: int = 42,
    n_gpu_layers: int = 999,
    timeout_sec: int = 1800,
) -> Tuple[Optional[dict], Dict[str, Any]]:
    '''Generate one MCQ JSON and return it as a dict (None on failure).'''

    cmd = [
        LLAMA_BIN,
        "-m", model_path,
        "-c", str(ctx_size),
        "-n", str(max_new_tokens),
        "-s", str(seed),
        "--temp", str(temperature),
        "--top-p", str(top_p),
        "-ngl", str(n_gpu_layers),
        "-p", prompt,
    ]

    if SUPPORT_JSON_SCHEMA_FILE:
        cmd += ["-jf", schema_path]
    else:
        pass

    if SUPPORT_NO_DISPLAY_PROMPT:
        cmd += ["--no-display-prompt"]
    if SUPPORT_NO_PERF:
        cmd += ["--no-perf"]
    if SUPPORT_SIMPLE_IO:
        cmd += ["--simple-io"]
    if SUPPORT_LOG_DISABLE:
        cmd += ["--log-disable"]
    if SUPPORT_SINGLE_TURN:
        cmd += ["-st"]

    t0 = time.time()
    p = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout_sec)
    dt = time.time() - t0

    out = (p.stdout or "").strip()
    err = (p.stderr or "").strip()

    obj = _extract_json_object(out)
    meta = {
        "seconds": dt,
        "returncode": p.returncode,
        "stderr_head": err[:400],
        "stdout_head": out[:200],
        "cmd_head": " ".join(cmd[:10]) + " ...",
        "constraints": "json_schema_file" if SUPPORT_JSON_SCHEMA_FILE else "none",
    }

    if obj is None:
        meta["parse_ok"] = False
        return None, meta

    try:
        validate(instance=obj, schema=mcq_schema)
        meta["schema_ok"] = True
    except ValidationError as e:
        meta["schema_ok"] = False
        meta["schema_error"] = str(e)[:400]

    meta["parse_ok"] = True
    return obj, meta


In [ ]:
import textwrap, re
from typing import List, Dict

def _shorten(s: str, max_len: int = 900) -> str:
    s = re.sub(r"\s+", " ", (s or "")).strip()
    if len(s) <= max_len:
        return s
    return s[:max_len-3] + "..."

def build_prompt_for_topic(
    doc_text: str,
    topic: str,
    q_index: int,
    n_questions: int,
    prior_items: List[Dict],
    run_mode: str,
) -> str:
    MAX_CHARS = 14000
    context = doc_text[:MAX_CHARS]

    asked_questions = [it.get("Question","") for it in prior_items if it.get("Question")]
    asked_block = "\n".join([f"- {_shorten(q, 220)}" for q in asked_questions][-12:])

    topic_hint = topic if topic else "Pick a distinct concept from the lecture material."

    return textwrap.dedent(f'''
    You are an instructor creating high-quality multiple-choice questions (MCQs) for students.

    OUTPUT FORMAT (STRICT):
    - Output must be a single JSON object only (no markdown, no commentary, no code fence).
    - Create exactly ONE MCQ with 5 choices: OptionA..OptionE.
    - CorrectOption must be one of: A, B, C, D, E (single letter).
    - SourcePages must be included (e.g., "p.1" or "p.1-2").

    CONTENT RULES (STRICT):
    - Use ONLY the provided lecture material as the source. Do NOT add facts not in the material.
    - The question must be answerable from the material.
    - The explanation must justify the correct option using the material.

    QUALITY RULES (VERY IMPORTANT):
    - The MCQ must be distinct from previously asked questions.
    - Avoid generating definition-only questions repeatedly.
    - Ensure the 5 options are mutually exclusive and NOT duplicates.
    - Ensure there is EXACTLY ONE correct answer.
    - Avoid algebraically equivalent distractors (e.g., do NOT include both "-Σ p log2 p" and "Σ p log2(1/p)" in the same question).

    RUN MODE:
    - {run_mode}
      (If 'deploy': your output will be automatically checked; if it fails (duplicates/ambiguity), it will be regenerated.)

    TARGET TOPIC (must focus on this):
    {topic_hint}

    PREVIOUSLY ASKED QUESTIONS (do NOT repeat these):
    {asked_block if asked_block else "- (none)"}

    LECTURE MATERIAL:
    {context}

    TASK:
    Generate MCQ #{q_index+1} out of {n_questions}.
    ''').strip()


In [ ]:
import time, re, json, difflib
from tqdm import tqdm
from typing import Tuple, List, Dict, Any, Optional
import numpy as np
import math
from collections import Counter

def normalize_correct_option(x: str) -> str:
    x = (x or "").strip()
    m = re.match(r"^([A-Ea-e])", x)
    if m:
        return m.group(1).upper()
    return x

def _norm_for_sim(s: str) -> str:
    s = (s or "").lower()
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"[^a-z0-9\s\?\!\.,\-\(\)\:]", "", s)
    return s

def sim_ratio(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, _norm_for_sim(a), _norm_for_sim(b)).ratio()

def canon_option_text(s: str) -> str:
    s = (s or "").lower()
    s = s.replace("−", "-")
    s = s.replace("*", "")
    s = re.sub(r"\s+", "", s)
    if "=" in s:
        s = s.split("=")[-1]
    s = s.replace("(bits)", "").replace("bits", "").replace("(nats)", "").replace("nats", "")
    return s.strip()

import ast, math

_ALLOWED_FUNCS = {
    "log2": math.log2,
    "ln": math.log,
    "log": math.log,
    "exp": math.exp,
    "sqrt": math.sqrt,
}
_ALLOWED_CONSTS = {
    "pi": math.pi,
    "e": math.e,
}
_ALLOWED_NAMES = {**_ALLOWED_FUNCS, **_ALLOWED_CONSTS}

_ALLOWED_NODES = (
    ast.Expression, ast.BinOp, ast.UnaryOp, ast.Call, ast.Load, ast.Name, ast.Constant,
    ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.USub, ast.UAdd,
)

def try_eval_const_expr(raw: str) -> Optional[float]:
    s = (raw or "").strip()
    if not s:
        return None
    s = s.replace("−","-")
    s = re.sub(r"\(bits\)|\bbits\b|\(nats\)|\bnats\b", "", s, flags=re.I)
    if "=" in s:
        s = s.split("=")[-1].strip()
    if any(ch in s for ch in ["Σ","sum", "p(x)", "p_i", "V1", "V2", "T", "k_B", "beta", "β", "Ω", "Omega", "Δ", "Delta", "E_i"]):
        return None
    try:
        tree = ast.parse(s, mode="eval")
    except Exception:
        return None

    for node in ast.walk(tree):
        if not isinstance(node, _ALLOWED_NODES):
            return None
        if isinstance(node, ast.Name) and node.id not in _ALLOWED_NAMES:
            return None
        if isinstance(node, ast.Call):
            if not isinstance(node.func, ast.Name):
                return None
            if node.func.id not in _ALLOWED_FUNCS:
                return None

    try:
        val = eval(compile(tree, "<const_expr>", "eval"), {"__builtins__": {}}, _ALLOWED_NAMES)
        if isinstance(val, (int, float)) and math.isfinite(val):
            return float(val)
    except Exception:
        return None
    return None

def has_duplicate_options_numeric_const(opts: List[str], tol: float = 1e-9) -> bool:
    vals = [try_eval_const_expr(o) for o in opts]
    for i in range(len(vals)):
        if vals[i] is None:
            continue
        for j in range(i+1, len(vals)):
            if vals[j] is None:
                continue
            if abs(vals[i] - vals[j]) <= tol:
                return True
    return False


def parse_and_eval_option_px(opt: str, p: np.ndarray) -> Optional[float]:
    #   - - sum_x p(x) log2 p(x)
    #   - sum_x p(x) log2 (1/p(x))
    #   - - sum_x p(x) log p(x)
    #   - log2 sum_x p(x)
    s = (opt or "").lower()
    if "=" in s:
        s = s.split("=")[-1]
    s = s.replace("(bits)", "").replace("bits", "").replace("(nats)", "").replace("nats", "")
    s = re.sub(r"\s+", "", s)

    def log2(x): return math.log(x, 2)
    def ln(x): return math.log(x)

    # log2 sum_x p(x) -> log2(1)=0
    if re.search(r"log2sum_?xp\(x\)", s) or "log2sumxp(x)" in s:
        return log2(float(p.sum()))

    if "p(x)/log2p(x)" in s:
        neg = s.startswith("-")
        val = float(np.sum(np.array([pi / log2(pi) for pi in p])))
        return -val if neg else val

    # sum p log2(1/p)
    if re.search(r"log2\(1/p\(x\)\)", s) or "log2(1/p(x))" in s:
        neg = s.startswith("-")
        val = float(np.sum(p * np.array([log2(1.0/pi) for pi in p])))
        return -val if neg else val

    # sum p log2 p
    if "log2p(x)" in s:
        neg = s.startswith("-")
        val = float(np.sum(p * np.array([log2(pi) for pi in p])))
        return -val if neg else val

    # sum p log p (natural)
    if re.search(r"logp\(x\)", s) or "lnp(x)" in s:
        neg = s.startswith("-")
        val = float(np.sum(p * np.array([ln(pi) for pi in p])))
        return -val if neg else val

    return None
def options_equivalent_px(opt1: str, opt2: str, trials: int = 5, k: int = 5) -> bool:
    for _ in range(trials):
        x = np.random.rand(k)
        p = x / x.sum()
        v1 = parse_and_eval_option_px(opt1, p)
        v2 = parse_and_eval_option_px(opt2, p)
        if v1 is None or v2 is None:
            return False
        if abs(v1 - v2) > 1e-6:
            return False
    return True

def validate_item(item: Dict[str, Any], prior_items: List[Dict[str, Any]]) -> Tuple[bool, List[str]]:
    problems = []

    q = item.get("Question","") or ""
    if q.strip() == "" or q.strip() == "[FAILED_TO_GENERATE]":
        problems.append("empty_question")

    # CorrectOption
    c = normalize_correct_option(item.get("CorrectOption",""))
    if c not in list("ABCDE"):
        problems.append(f"invalid_correct_option:{item.get('CorrectOption')}")
    item["CorrectOption"] = c

    # Options
    opts = [item.get("OptionA",""), item.get("OptionB",""), item.get("OptionC",""), item.get("OptionD",""), item.get("OptionE","")]
    if any((o or "").strip()=="" for o in opts):
        problems.append("empty_option")

    canon = [canon_option_text(o) for o in opts]
    if len(set(canon)) < 5:
        problems.append("duplicate_options_canonical")

    if ENABLE_PX_EQUIV_TEST and any("p(x)" in (o or "").lower() for o in opts):
        corr_map = {"A":0,"B":1,"C":2,"D":3,"E":4}
        if c in corr_map:
            corr_opt = opts[corr_map[c]]
            for j,other in enumerate(opts):
                if j == corr_map[c]:
                    continue
                if canon_option_text(other) == canon_option_text(corr_opt):
                    problems.append("another_option_equivalent_to_correct (canonical)")
                    break
                if options_equivalent_px(corr_opt, other, trials=PX_EQUIV_TRIALS):
                    problems.append("another_option_equivalent_to_correct (px_equiv_test)")
                    break

    for prev in prior_items:
        prev_q = prev.get("Question","") or ""
        if prev_q.strip()=="":
            continue
        if sim_ratio(q, prev_q) >= DUP_SIM_THRESHOLD:
            problems.append("duplicate_question_fuzzy")
            break

    ok = (len(problems) == 0)
    return ok, problems

def extract_excerpt_by_source_pages(doc_text: str, source_pages: str, max_chars: int = 1600) -> str:
    """Use SourcePages (e.g., 'p.1-2') to extract a nearby snippet from doc_text."""
    s = (source_pages or "")
    nums = re.findall(r"p\.(\d+)", s)
    if not nums:
        return doc_text[:max_chars]
    a = int(nums[0])
    b = int(nums[-1])
    if b < a:
        a, b = b, a
    out = []
    for k in range(a, b+1):
        marker = f"(p.{k})"
        idx = doc_text.find(marker)
        if idx < 0:
            continue
        nxt = doc_text.find(f"(p.{k+1})", idx+1)
        chunk = doc_text[idx: (nxt if nxt>0 else idx+max_chars)]
        out.append(chunk)
        if sum(len(x) for x in out) >= max_chars:
            break
    txt = "\n\n".join(out).strip()
    return txt[:max_chars]

def build_verifier_prompt(item: Dict[str, Any], source_excerpt: str) -> str:
    q = item.get("Question","")
    opts = {
        "A": item.get("OptionA",""),
        "B": item.get("OptionB",""),
        "C": item.get("OptionC",""),
        "D": item.get("OptionD",""),
        "E": item.get("OptionE",""),
    }
    return textwrap.dedent(f"""    You are a strict examiner.

    Task:
    - Determine which answer choices (A–E) are correct.
    - IMPORTANT: In well-formed MCQs, exactly ONE option should be correct.
    - If multiple options are correct or none is correct, still list all correct options you found.

    Return ONLY a JSON object that matches this schema:
    {{
      "correct_options": ["A"],
      "notes": "short reason (optional)"
    }}

    Source excerpt (may be partial):
    {source_excerpt}

    Question:
    {q}

    Choices:
    A) {opts["A"]}
    B) {opts["B"]}
    C) {opts["C"]}
    D) {opts["D"]}
    E) {opts["E"]}
    """)

def verifier_check_unique_correct(
    item: Dict[str, Any],
    doc_text: str,
    model_path: str,
    seed: int,
) -> Tuple[bool, Dict[str, Any]]:
    excerpt = extract_excerpt_by_source_pages(doc_text, item.get("SourcePages","p.?"), max_chars=1600)
    prompt = build_verifier_prompt(item, excerpt)
    obj, meta = llama_generate_json_one(
        prompt=prompt,
        model_path=model_path,
        schema_path=VERIFIER_SCHEMA_PATH,
        ctx_size=min(CTX_SIZE, 4096),
        max_new_tokens=VERIFIER_MAX_NEW_TOKENS,
        temperature=0.0,
        top_p=1.0,
        seed=seed,
        n_gpu_layers=N_GPU_LAYERS,
    )
    # parse result
    corrects = []
    if obj and isinstance(obj.get("correct_options", None), list):
        for x in obj["correct_options"]:
            if not isinstance(x, str):
                continue
            m = re.match(r"^([A-Ea-e])", x.strip())
            if m:
                corrects.append(m.group(1).upper())
    corrects = sorted(set(corrects))
    out = {"verifier_correct_options": corrects, "verifier_raw": obj}
    # well-formed if exactly one correct option
    ok = (len(corrects) == 1)
    return ok, out


def generate_one_question_for_topic(
    doc_text: str,
    topic: str,
    qi: int,
    n_questions: int,
    model_path: str,
    seed: int,
    prior_items: List[Dict[str, Any]],
) -> Tuple[Dict[str, Any], Dict[str, Any]]:
    tries = 0
    last_meta = None
    last_obj = None
    last_problems = None

    while tries < MAX_TRIES_PER_QUESTION:
        prompt = build_prompt_for_topic(
            doc_text=doc_text,
            topic=topic,
            q_index=qi,
            n_questions=n_questions,
            prior_items=prior_items,
            run_mode=RUN_MODE,
        )

        obj = None
        meta = None

        for r in range(RETRY_MAX):
            obj, meta = llama_generate_json_one(
                prompt=prompt,
                model_path=model_path,
                schema_path=MCQ_SCHEMA_PATH,
                ctx_size=CTX_SIZE,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                top_p=TOP_P,
                seed=seed + tries,
                n_gpu_layers=N_GPU_LAYERS,
            )
            if obj is not None and meta.get("schema_ok", True):
                break
            obj = None

        last_meta = meta or {}
        last_obj = obj
        if obj is None:
            tries += 1
            continue

        obj["CorrectOption"] = normalize_correct_option(obj.get("CorrectOption",""))
        if not obj.get("ID"):
            obj["ID"] = f"Q{qi+1:02d}"
        if not obj.get("Category"):
            obj["Category"] = "MCQ"
        if not obj.get("SourcePages"):
            obj["SourcePages"] = "p.?"
        obj["FocusTopic"] = topic

        ok, problems = validate_item(obj, prior_items)
        last_problems = problems

        if RUN_MODE == "eval":
            last_meta["qc_ok"] = ok
            last_meta["qc_problems"] = problems
            last_meta["tries"] = tries + 1
            return obj, last_meta

        if ok:
            last_meta["qc_ok"] = True
            last_meta["qc_problems"] = []
            last_meta["tries"] = tries + 1
            return obj, last_meta

        tries += 1

    fallback = last_obj if last_obj is not None else {
        "ID": f"Q{qi+1:02d}",
        "Category": "MCQ",
        "SourcePages": "p.?",
        "Question": "[FAILED_TO_GENERATE]",
        "OptionA": "", "OptionB": "", "OptionC": "", "OptionD": "", "OptionE": "",
        "CorrectOption": "",
        "Explanation": "Generation failed (max tries reached).",
        "FocusTopic": topic,
    }
    last_meta = last_meta or {}
    last_meta["qc_ok"] = False
    last_meta["qc_problems"] = last_problems or ["failed_or_max_tries"]
    last_meta["tries"] = MAX_TRIES_PER_QUESTION
    return fallback, last_meta

def generate_mcqs_for_one_pdf(pdf_path: str, model_path: str, seeds: List[int]):
    def pdf_to_text_with_pages(pdf_path: str) -> str:
        d = fitz.open(pdf_path)
        ps = []
        for i, page in enumerate(d):
            txt = page.get_text("text")
            txt = re.sub(r"\s+\n", "\n", txt)
            ps.append(f"(p.{i+1})\n{txt.strip()}")
        return "\n\n".join(ps).strip()

    results_all = []
    for seed in seeds:
        doc_text = pdf_to_text_with_pages(pdf_path)
        topic_plan = get_topic_plan(pdf_path, doc_text, N_QUESTIONS)

        t0 = time.time()
        items = []
        metas = []

        for qi in tqdm(range(N_QUESTIONS), desc=f"Generating (seed={seed})"):
            topic = topic_plan[qi] if qi < len(topic_plan) else ""
            obj, meta = generate_one_question_for_topic(
                doc_text=doc_text,
                topic=topic,
                qi=qi,
                n_questions=N_QUESTIONS,
                model_path=model_path,
                seed=seed,
                prior_items=items,
            )
            items.append(obj)
            metas.append(meta)

        dt = time.time() - t0
        results_all.append({
            "pdf_path": pdf_path,
            "seed": seed,
            "seconds_total": dt,
            "topic_plan": topic_plan,
            "items": items,
            "metas": metas,
        })
    return results_all

if RUN_BATCH:
    RUNS = []
    for p in BATCH_PDF_LIST:
        RUNS.extend(generate_mcqs_for_one_pdf(p, MODEL_PATH, seeds=BATCH_SEEDS))
else:
    RUNS = generate_mcqs_for_one_pdf(PDF_PATH, MODEL_PATH, seeds=[SEED])

print("✅ runs:", len(RUNS))
print("✅ example item keys:", list(RUNS[0]["items"][0].keys()))


In [ ]:
import pandas as pd
import numpy as np
import re, difflib
from collections import Counter
from typing import List, Dict, Any, Tuple

import ast, math
_ALLOWED_FUNCS = {
    "log2": math.log2,
    "ln": math.log,
    "log": math.log,
    "exp": math.exp,
    "sqrt": math.sqrt,
}
_ALLOWED_CONSTS = {"pi": math.pi, "e": math.e}
_ALLOWED_NAMES = {**_ALLOWED_FUNCS, **_ALLOWED_CONSTS}
_ALLOWED_NODES = (
    ast.Expression, ast.BinOp, ast.UnaryOp, ast.Call, ast.Load, ast.Name, ast.Constant,
    ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.USub, ast.UAdd,
)
def try_eval_const_expr(raw: str):
    s = (raw or "").strip()
    if not s:
        return None
    s = s.replace("−","-")
    s = re.sub(r"\(bits\)|\bbits\b|\(nats\)|\bnats\b", "", s, flags=re.I)
    if "=" in s:
        s = s.split("=")[-1].strip()
    if any(ch in s for ch in ["Σ","sum", "p(x)", "p_i", "V1", "V2", "T", "k_B", "beta", "β", "Ω", "Omega", "Δ", "Delta", "E_i"]):
        return None
    try:
        tree = ast.parse(s, mode="eval")
    except Exception:
        return None
    for node in ast.walk(tree):
        if not isinstance(node, _ALLOWED_NODES):
            return None
        if isinstance(node, ast.Name) and node.id not in _ALLOWED_NAMES:
            return None
        if isinstance(node, ast.Call):
            if not isinstance(node.func, ast.Name) or node.func.id not in _ALLOWED_FUNCS:
                return None
    try:
        val = eval(compile(tree, "<const_expr>", "eval"), {"__builtins__": {}}, _ALLOWED_NAMES)
        if isinstance(val, (int,float)) and math.isfinite(val):
            return float(val)
    except Exception:
        return None
    return None

def has_duplicate_options_numeric_const(opts: List[str], tol: float = 1e-9) -> bool:
    vals = [try_eval_const_expr(o) for o in opts]
    for i in range(len(vals)):
        if vals[i] is None:
            continue
        for j in range(i+1, len(vals)):
            if vals[j] is None:
                continue
            if abs(vals[i]-vals[j]) <= tol:
                return True
    return False


def extract_numbers(s: str):
    if not s:
        return []
    nums = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", s)
    out = []
    for n in nums:
        try:
            out.append(float(n))
        except Exception:
            pass
    return out

def _norm_for_sim2(s: str) -> str:
    s = (s or "").lower()
    s = re.sub(r"\s+", " ", s).strip()
    return s

def sim_ratio2(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, _norm_for_sim2(a), _norm_for_sim2(b)).ratio()

def canon_option_text2(s: str) -> str:
    s = (s or "").lower()
    s = s.replace("−", "-").replace("*","")
    s = re.sub(r"\s+", "", s)
    if "=" in s:
        s = s.split("=")[-1]
    s = s.replace("(bits)", "").replace("bits","").replace("(nats)", "").replace("nats","")
    return s.strip()

def auto_check_items_v2(items: List[Dict[str, Any]], dup_threshold: float = 0.92) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
    issues = []

    q_texts = [ (it.get("ID",""), (it.get("Question","") or "").strip()) for it in items ]
    q_only = [q for _,q in q_texts]

    counts = Counter([q.lower() for q in q_only if q])
    dup_exact = [q for q,c in counts.items() if c>1]
    for q in dup_exact:
        ids = [qid for qid,qq in q_texts if qq.lower()==q]
        issues.append({"type":"duplicate_question_exact","ids":ids,"detail":q[:140]})

    near_pairs = []
    for i in range(len(q_texts)):
        for j in range(i+1, len(q_texts)):
            qi, qj = q_texts[i][1], q_texts[j][1]
            if not qi or not qj:
                continue
            r = sim_ratio2(qi, qj)
            if r >= dup_threshold:
                near_pairs.append((q_texts[i][0], q_texts[j][0], float(r)))
    for a,b,r in near_pairs:
        issues.append({"type":"duplicate_question_fuzzy_pair","pair":[a,b],"ratio":r})

    for it in items:
        qid = it.get("ID","")
        opts = [it.get("OptionA",""),it.get("OptionB",""),it.get("OptionC",""),it.get("OptionD",""),it.get("OptionE","")]
        if any((o or "").strip()=="" for o in opts):
            issues.append({"type":"empty_option","id":qid})

        canon = [canon_option_text2(o) for o in opts]
        if len(set(canon)) < 5:
            issues.append({"type":"duplicate_options_canonical","id":qid})

        if ENABLE_CONST_EQUIV_CHECK and has_duplicate_options_numeric_const(opts, tol=CONST_EQUIV_TOL):
            issues.append({"type":"duplicate_options_numeric_const","id":qid})

        c = (it.get("CorrectOption","") or "").strip().upper()
        if c not in list("ABCDE"):
            issues.append({"type":"invalid_correct_option","id":qid,"value":c})

        if ENABLE_PX_EQUIV_TEST and any("p(x)" in (o or "").lower() for o in opts) and c in list("ABCDE"):
            corr_idx = "ABCDE".index(c)
            corr_opt = opts[corr_idx]
            for j,other in enumerate(opts):
                if j==corr_idx:
                    continue
                if canon_option_text2(other) == canon_option_text2(corr_opt):
                    issues.append({"type":"another_option_equivalent_to_correct","id":qid,"method":"canonical","other":"ABCDE"[j]})
                    break
                if options_equivalent_px(corr_opt, other, trials=PX_EQUIV_TRIALS):
                    issues.append({"type":"another_option_equivalent_to_correct","id":qid,"method":"px_equiv_test","other":"ABCDE"[j]})
                    break

        exp = it.get("Explanation","") or ""
        corr_map = {"A":"OptionA","B":"OptionB","C":"OptionC","D":"OptionD","E":"OptionE"}
        corr_text = it.get(corr_map.get(c,""),"") if c in corr_map else ""
        corr_nums = extract_numbers(corr_text)
        exp_nums = extract_numbers(exp)
        if corr_nums and exp_nums:
            if not any(any(abs(a-b) < 1e-9 for b in exp_nums) for a in corr_nums):
                issues.append({"type":"numeric_mismatch_suspect","id":qid,"corr_nums":corr_nums[:3],"exp_nums":exp_nums[:5]})

        round_words = ["round", "rounded", "approx", "approximately", "≈"]
        if any(w in exp.lower() for w in round_words) and not any(w in (it.get("Question","").lower()) for w in ["round","approx","approximately"]):
            issues.append({"type":"rounding_instruction_missing_suspect","id":qid})

    q_unique_exact = len(set([q.lower() for q in q_only if q]))
    q_total = len([q for q in q_only if q])
    qc_meta = {
        "n_questions_total": q_total,
        "n_questions_unique_exact": q_unique_exact,
        "n_questions_duplicate_occurrences_exact": max(0, q_total - q_unique_exact),
        "n_fuzzy_duplicate_pairs": len(near_pairs),
        "dup_threshold": dup_threshold,
    }
    return issues, qc_meta

CHECK_ROWS = []
for run in RUNS:
    its = run["items"]
    issues, qc_meta = auto_check_items_v2(its, dup_threshold=DUP_SIM_THRESHOLD)
    run["issues"] = issues
    run["qc_meta"] = qc_meta

    tries_list = [m.get("tries",1) for m in run.get("metas",[])]
    tries_total = int(sum([t for t in tries_list if isinstance(t,(int,float))]))

    qc_ok_list = [m.get("qc_ok", None) for m in run.get("metas",[])]
    qc_ok_rate = None
    if qc_ok_list and all(x is not None for x in qc_ok_list):
        qc_ok_rate = float(sum(1 for x in qc_ok_list if x) / len(qc_ok_list))

    CHECK_ROWS.append({
        "pdf_path": run["pdf_path"],
        "seed": run["seed"],
        "seconds_total": run["seconds_total"],
        "n_items": len(its),
        "n_failed": sum(1 for x in its if x.get("Question") == "[FAILED_TO_GENERATE]"),
        "n_issues": len(issues),
        "issue_types": ", ".join(sorted(set(i["type"] for i in issues))) if issues else "",
        "unique_q_exact": qc_meta.get("n_questions_unique_exact"),
        "dup_occ_exact": qc_meta.get("n_questions_duplicate_occurrences_exact"),
        "fuzzy_dup_pairs": qc_meta.get("n_fuzzy_duplicate_pairs"),
        "tries_total": tries_total,
        "qc_ok_rate": qc_ok_rate,
    })

summary_df = pd.DataFrame(CHECK_ROWS)
summary_df


In [ ]:
# Export artifacts per run (including run_manifest.json)
import os, json, pathlib, hashlib, subprocess, platform
import pandas as pd
from datetime import datetime, timezone

OUT_ROOT = pathlib.Path("/content/output_mcq")
OUT_ROOT.mkdir(parents=True, exist_ok=True)
ts = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%SZ")
OUT_DIR = OUT_ROOT / ts
OUT_DIR.mkdir(parents=True, exist_ok=True)

def items_to_questionbank_df(items):
    rows = []
    for it in items:
        rows.append({
            "ID": it.get("ID",""),
            "Category": it.get("Category",""),
            "FocusTopic": it.get("FocusTopic",""),
            "SourcePages": it.get("SourcePages",""),
            "Question": it.get("Question",""),
            "OptionA": it.get("OptionA",""),
            "OptionB": it.get("OptionB",""),
            "OptionC": it.get("OptionC",""),
            "OptionD": it.get("OptionD",""),
            "OptionE": it.get("OptionE",""),
            "CorrectOption": it.get("CorrectOption",""),
            "Explanation": it.get("Explanation",""),
        })
    return pd.DataFrame(rows)

def sha256_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1024*1024), b""):
            h.update(chunk)
    return h.hexdigest()

def try_git_commit(repo_dir: str):
    try:
        return subprocess.check_output(
            ["git", "-C", repo_dir, "rev-parse", "HEAD"],
            stderr=subprocess.DEVNULL,
        ).decode("utf-8").strip()
    except Exception:
        return None

LLAMA_CPP_COMMIT = try_git_commit(str(LLAMA_DIR)) if "LLAMA_DIR" in globals() else None

def build_run_manifest(run, run_dir: pathlib.Path):
    pdf_basename = os.path.basename(run["pdf_path"])
    seed = int(run.get("seed", -1))
    model_path = globals().get("MODEL_PATH", None)

    # model hashes are optional (model file may be large / not bundled)
    model_sha256 = None
    model_filename = None
    if model_path and os.path.exists(model_path):
        model_filename = os.path.basename(model_path)
        try:
            model_sha256 = sha256_file(model_path)
        except Exception:
            model_sha256 = None

    # artifact checksums (small files)
    q_json = run_dir / "questions.json"
    qb_csv = run_dir / "QuestionBank.csv"
    issues_json = run_dir / "auto_check_issues.json"

    manifest = {
        "run_id": run_dir.name,
        "pdf_basename": pdf_basename,
        "seed": seed,
        "timestamps": {
            "generated_utc": datetime.now(timezone.utc).replace(microsecond=0).isoformat()
        },
        "environment": {
            "platform": "Google Colab" if os.path.exists("/content") else "local",
            "python": platform.python_version(),
            "os": platform.platform(),
        },
        "toolchain": {
            "llama_cpp_repo": "https://github.com/ggml-org/llama.cpp",
            "llama_cpp_commit": LLAMA_CPP_COMMIT,
        },
        "model": {
            "mode": globals().get("MODEL_MODE", None),
            "hf_repo_id": globals().get("HF_REPO_ID", None),
            "hf_filename": globals().get("HF_FILENAME", None),
            "local_path": model_path,
            "gguf_filename": model_filename,
            "gguf_sha256": model_sha256,
        },
        "decoding": {
            "ctx_size": globals().get("CTX_SIZE", None),
            "n_gpu_layers": globals().get("N_GPU_LAYERS", None),
            "temperature": globals().get("TEMPERATURE", None),
            "top_p": globals().get("TOP_P", None),
            "max_new_tokens": globals().get("MAX_NEW_TOKENS", None),
        },
        "qc": {
            "n_questions_target": globals().get("N_QUESTIONS", None),
            "retry_max": globals().get("RETRY_MAX", None),
            "dup_similarity_threshold": globals().get("DUP_SIM_THRESHOLD", None),
            "enable_const_equiv_check": globals().get("ENABLE_CONST_EQUIV_CHECK", None),
            "const_equiv_tol": globals().get("CONST_EQUIV_TOL", None),
            "enable_px_equiv_test": globals().get("ENABLE_PX_EQUIV_TEST", None),
            "px_equiv_trials": globals().get("PX_EQUIV_TRIALS", None),
            "enable_unique_correct": globals().get("ENABLE_UNIQUE_CORRECT_CHECK", None),
        },
        "results": {
            "seconds_total": float(run.get("seconds_total", 0.0)),
            "n_items": int(len(run.get("items", []))),
            "warnings_count": int(len(run.get("issues", []))),
            "warning_types": sorted(set([i.get("type","") for i in run.get("issues", []) if i.get("type")])),
        },
        "files": {
            "questions_json": {"path": q_json.name, "sha256": sha256_file(str(q_json))} if q_json.exists() else None,
            "questionbank_csv": {"path": qb_csv.name, "sha256": sha256_file(str(qb_csv))} if qb_csv.exists() else None,
            "auto_check_issues_json": {"path": issues_json.name, "sha256": sha256_file(str(issues_json))} if issues_json.exists() else None,
        },
    }
    return manifest

all_run_artifacts = []

for run in RUNS:
    safe_pdf = os.path.basename(run["pdf_path"]).replace(".pdf","")
    run_tag = f"{safe_pdf}_seed{int(run['seed'])}"
    run_dir = OUT_DIR / run_tag
    run_dir.mkdir(parents=True, exist_ok=True)

    # save run-level artifacts
    items = run.get("items", [])
    with open(run_dir / "questions.json", "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

    with open(run_dir / "auto_check_issues.json", "w", encoding="utf-8") as f:
        json.dump(run.get("issues", []), f, ensure_ascii=False, indent=2)

    df = items_to_questionbank_df(items)
    df.to_csv(run_dir / "QuestionBank.csv", index=False, encoding="utf-8")

    # manifest
    manifest = build_run_manifest(run, run_dir)
    with open(run_dir / "run_manifest.json", "w", encoding="utf-8") as f:
        json.dump(manifest, f, ensure_ascii=False, indent=2)

    all_run_artifacts.append({
        "run_tag": run_tag,
        "questions_json": str(run_dir / "questions.json"),
        "questionbank_csv": str(run_dir / "QuestionBank.csv"),
        "issues_json": str(run_dir / "auto_check_issues.json"),
        "run_manifest_json": str(run_dir / "run_manifest.json"),
    })

# Save summary if available
summary_path = OUT_DIR / "runs_summary.csv"
if "summary_df" in globals():
    summary_df.to_csv(summary_path, index=False, encoding="utf-8")
else:
    pd.DataFrame(all_run_artifacts).to_csv(summary_path, index=False, encoding="utf-8")

print("✅ OUT_DIR:", OUT_DIR)
print("✅ summary:", summary_path)

pd.DataFrame(all_run_artifacts)


In [ ]:
# Optional: zip the output directory for download/sharing
import shutil

zip_path = shutil.make_archive(str(OUT_DIR), 'zip', root_dir=str(OUT_DIR))
print('Created:', zip_path)

# In Google Colab you may uncomment the following:
# from google.colab import files
# files.download(zip_path)
